# Proyecto MDS7203

**Benjamín Farías**

**Sebastián Sanhueza**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import gdown

from torchvision import transforms as T
from torchvision.ops import Permute
from torch.nn.utils import spectral_norm
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.notebook import tqdm
from matplotlib import gridspec

from .dataset import CaptchaDataset
from .model import HVAE
from .trainer import train_hvae, label_one_hot, plot_loss
from .generate import plot_samples, plot_samples_by_normal, plot_cond_samples

## Cargar Dataset

In [ ]:
# Descarga del dataset
gdown.download(id='1ANp5zuxm0UcWPf3ufKvebQIr22OaKX7x', output='captcha.zip', quiet=False)

In [ ]:
%%capture
!unzip captcha.zip

In [ ]:
# directorio donde se descargó el dataset
WORKDIR = '/content/'

# Carga del dataset utilizado
data = CaptchaDataset(WORKDIR + 'captchas/train', T.Compose([T.ToTensor()]))

loader = DataLoader(data, batch_size=64, shuffle=True)

In [ ]:
# Extracción de todas las etiquetas en la data de entrenamiento
train_labels = [element[1] for element in data]

# Identificación de caracteres únicos
labels_string = "".join(train_labels)
chars = sorted(list(set(labels_string)))

vocab_size = len(chars)

# Diccionario tipo caracter:índice
stoi = {char:index for index, char in enumerate(chars)}

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

## Sampleo

In [ ]:
# Descarga del modelo pre-entrenado
gdown.download(id='1wXnksXWeDPaBYq1pbJQMb-sqbWjIukWr', output='HVAE.pt', quiet=False)

In [ ]:
# Carga los parámetros entrenados del modelo
model = HVAE().to(device)
model.load('HVAE.pt')

In [ ]:
# Generar muestras de la distribución de z_T|x
plot_samples(32, model, loader, device=device)

In [ ]:
# Generar muestras de N(0, I) 
plot_samples_by_normal(32, model, device=device)

## Sampleo Condicional

### Entrenar modelo condicional

In [ ]:
model_cond = HVAE(inner_config=[], conditioned=True)

train_hvae(model_cond, loader, device=device, conditioned=True)

### Generar muestras

In [ ]:
n_samples = 5
label = ['26pzgu']
plot_cond_samples(n_samples, model_cond, loader, label, device=device)